## Fase 4: Limpieza de datos

Tenemos más de 100 columnas de las que sólo queremos las concernientes a sus datos de programación

Nos vamos a quedar con las siguientes columnas

get_languages()
allies, clan, comments, followers, following, github, honor, last seen, 
leaderboard position # eliminar el #
member since, total completed kata, total languages trained, translations, username, avg. satisfaction rating, 
contributed kata, created, data structures, data types, fundamentals, graphs, kata approvals, 
total collected, total completions, total stars, skills, linkedin


In [4]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display
import time

pd.options.display.max_columns = None

In [5]:
df = pd.read_csv('../output/df-codewars-2929x221.csv')
df.shape

(3939, 222)

In [6]:
df.head()

,Unnamed: 0,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,github,skills,4 kyu,5 kyu,6 kyu,7 kyu,8 kyu,best practice,best practice solutions,clever,clever solutions,completed kata,completed on 1st attempt,current streak,date,ended on,first completed,kata attempts,last completed,most number of days,avg. satisfaction rating,created,data types,puzzles,strings,total collected,total completions,total stars,linkedin,advanced language features,contributed kata,declarative programming,fundamentals,programming paradigms,regular expressions,avg. rank,utilities,data,databases,information systems,algorithms,character encodings,game boards,games,graphs,kata approvals,logic,mathematics,numbers,django,frameworks,algebra,geometry,arrays,basic language features,control flow,functions,metaprogramming,parsing,formats,prototypes,sparse arrays,babel,design principles,es2015,react,decryption,security,binary,binary search trees,computability theory,recursion,theoretical computer science,bits,bitwise operators,dates/time,functional programming,classes,3 kyu,authored kata,authored translations,search,data structures,object-oriented programming,variables,loops,lists,objects,arithmetic,applied computer science,machine learning,1 kyu,formatting,interpreters,2 kyu,sorting,optimization,data conversion,esoteric languages,angular,performance,integers,permutations,unicode,bugs,nodejs,queues,decoding,exception handling,sequences,statistics,networks,vectors,ascii,operators,hashes,map/reduce,refactoring,iterators,sets,cryptography,encryption,arguments,hacking holidays,filtering,trees,maps,design patterns,event handling,dynamic programming,rules,decimals,big integers,encoding,tables,physics,modules,best practices,conditional statements,observers,lambdas,computational science,state machines,inheritance,polymorphism,booleans,graphics,higher-order functions,immutability,mutability,linked lists,ranking,io,streams,expressions,properties,json,closures,reflection,dictionary,validation,reporting,ciphers
0,0,1.0,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0.0,2 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,JavaScript (5 kyu),376,94th,0.0,428.0,0.0,0.0,0,Feb 2017,"#33,216",0.0,Dec 2015,JavaScript,Unknown,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,5 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35,2.0,0 (0 approved),0.0,mr.tk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,47.0,0.0,0.0,0.0,0.0,Unknown,0.0,16.0,0.0,1 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,48.0,48.0,0.0,0.0,0.0,0.0,CoffeeScript (8 kyu),49,65th,8.0,0.0,0.0,0.0,0,May 2017,"#223,367",0.0,Aug 2016,Java,Theresa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,3.0,0 (0 approved),0.0,Caffeinatedbrew,https://github.com/Caffeinatedbrew,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

(3939, 222)